In [3]:
import json
import csv
import os
import glob
import pandas as pd

In [4]:
with open("../visData.json") as f:
    data = json.load(f)

In [5]:
file_names =  []
for item in data:
    file_names.append('..' + item['Highlight Data'])

In [6]:
combined_csv = pd.concat([pd.read_csv(f) for f in file_names ])

combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [21]:
json_data = pd.read_json('visData.json')
#renames and dropped unwanted columns
json_data = json_data.rename(columns = {'article_sha256': 'Article ID'}).drop(columns = ['Highlight Data','Plain Text','Visualization Link','Article Link', 'ID','articleHash'])
json_data

,Article ID,Title,Author,Date
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
1,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,"Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00
2,be0b18a87d4370fa579180ef26dcb7080598f27f9ec761...,SARS-CoV-2 Can Live on Plastic and Steel for 2...,Kerry Grens,2020-03-12 18:56:55+00:00
3,47990959103662e94e796d979018922afddc880fb4b867...,2005 CIA Report on Coronavirus Pandemic Discov...,Lyubov Stepushova,2020-03-17 18:56:55+00:00
4,3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4...,US military may have brought coronavirus to Wu...,Straits Times,2020-03-17 18:56:55+00:00


In [8]:
combined = pd.read_csv('combined_csv.csv')

In [9]:
combined.head(3)

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E0,Evidence,Alternative explanation,-1.868043,[],-1.0,-1.0,NaN
1,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E1,Evidence,Open to evidence,-1.657269,[],-1.0,-1.0,NaN
2,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H0,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN


In [89]:
combined['Start, End'] = combined[['Start','End']].apply(tuple,axis = 1)

In [90]:
#merge both csv and json dataframes together
full_data = pd.merge(left = combined, right = json_data, on = 'Article ID')
#whats Target Text?
full_data.head()

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text,"Start,End","Start, End",Title,Author,Date
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E0,Evidence,Alternative explanation,-1.868043,[],-1.0,-1.0,NaN,"(-1.0, -1.0)","(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
1,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E1,Evidence,Open to evidence,-1.657269,[],-1.0,-1.0,NaN,"(-1.0, -1.0)","(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
2,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H0,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)","(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
3,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H1,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)","(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
4,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H2,Holistic,Vague Sourcing,-2.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)","(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00


In [91]:
#Changes Date from DateType object to string type
full_data.Date = full_data.Date.astype(str)
data = full_data.groupby(['Article ID', 'Credibility Indicator Category','Credibility Indicator Name','Credibility Indicator ID','Start,End']).agg(lambda x:list(x)[0])





In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 118 entries, (3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4ec66b45681c86c9301, Evidence, Open to evidence, E0, (-1.0, -1.0)) to (be0b18a87d4370fa579180ef26dcb7080598f27f9ec76181f2cfd851f320da06, Probability, Open to evidence, P1, (2274.0, 2496.0))
Data columns (total 9 columns):
Points                         118 non-null float64
Indices of Label in Article    118 non-null object
Start                          118 non-null float64
End                            118 non-null float64
target_text                    36 non-null object
Start, End                     118 non-null object
Title                          118 non-null object
Author                         118 non-null object
Date                           118 non-null object
dtypes: float64(3), object(6)
memory usage: 10.2+ KB


In [93]:
data['Date'] = pd.to_datetime(data['Date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 118 entries, (3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4ec66b45681c86c9301, Evidence, Open to evidence, E0, (-1.0, -1.0)) to (be0b18a87d4370fa579180ef26dcb7080598f27f9ec76181f2cfd851f320da06, Probability, Open to evidence, P1, (2274.0, 2496.0))
Data columns (total 9 columns):
Points                         118 non-null float64
Indices of Label in Article    118 non-null object
Start                          118 non-null float64
End                            118 non-null float64
target_text                    36 non-null object
Start, End                     118 non-null object
Title                          118 non-null object
Author                         118 non-null object
Date                           118 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(3), object(5)
memory usage: 10.2+ KB


In [95]:
data

Points  \
Article ID                                         Credibility Indicator Category Credibility Indicator Name Credibility Indicator ID Start,End                    
3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4e... Evidence                       Open to evidence           E0                       (-1.0, -1.0)     -1.000000   
                                                   Holistic                       Qualified Source           H1                       (-1.0, -1.0)      2.000000   
                                                                                                             H2                       (-1.0, -1.0)      1.000000   
                                                                                                             H3                       (-1.0, -1.0)      2.000000   
                                                                                                             H4                       (-1.0, -1.0)      2.000000   
...                                                                                                                                                          ...   
be0b18a87d4370fa579180ef26dcb7080598f27f9ec7618... Holistic                       Qualified Source           H2                       (-1.0, -1.0)      2.000000   
                                                                                                             H3                       (-1.0, -1.0)      1.000000   
                                                                                  Vague Sourcing             H4                       (-1.0, -1.0)     -2.000000   
                                                   Probability                    Acknowledges uncertainty   P0                       (2204.0, 2389.0)  0.757298   
                                                                                  Open to evidence           P1                       (2274.0, 2496.0)  0.333333   

                                                                                                                                                                              Indices of Label in Article  \
Article ID                                         Credibility Indicator Category Credibility Indicator Name Credibility Indicator ID Start,End                                                             
3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4e... Evidence                       Open to evidence           E0                       (-1.0, -1.0)                                                     []   
                                                   Holistic                       Qualified Source           H1                       (-1.0, -1.0)                                                     []   
                                                                                                             H2                       (-1.0, -1.0)                                                     []   
                                                                                                             H3                       (-1.0, -1.0)                                                     []   
                                                                                                             H4                       (-1.0, -1.0)                                                     []   
...                                                                                                                                                                                                   ...   
be0b18a87d4370fa579180ef26dcb7080598f27f9ec7618... Holistic                       Qualified Source           H2                       (-1.0, -1.0)                                                     []   
                                                                                                             H3                       (-1.0, -1.0)                              